In [1]:
# import
import torch
from transformers import BertConfig, BertForPreTraining
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil
from icecream import ic
from tqdm import tqdm_notebook as tqdm 

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

from transformers import BertModel

In [2]:
dic = {}

with open("./BERT-base_mecab-ipadic-bpe-32k/vocab.txt",  "r", encoding="utf-8" ) as f:
    vocab = f.read()
    for id, word in enumerate(vocab.split('\n')):
        dic[word] = id
        
text = "[CLS] 私 は 犬 が 好き 。 [SEP]"

x = [dic[w] for w in text.split()]

In [3]:
x = torch.LongTensor(x).unsqueeze(0)



In [4]:
model = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")

In [5]:
a = model(x)

In [6]:
a[0].shape

torch.Size([1, 8, 768])

In [7]:
a[0][0][3]

tensor([-1.2636e-01, -1.2734e-01,  4.1125e-01, -3.7946e-02,  3.5671e-01,
         5.9472e-01, -1.2080e-02, -2.0193e-01,  6.1377e-01,  2.1312e-01,
        -7.4529e-02,  2.0608e-02, -4.5949e-01,  2.5509e-01, -2.4717e-01,
        -9.4191e-02,  1.1410e-01, -2.2248e-01,  3.5165e-01,  4.6572e-01,
        -3.8768e-01, -8.5532e-02,  2.5526e-01,  2.6166e-01, -3.1824e-01,
         8.1794e-02, -2.4993e-01,  2.9269e-01,  7.9955e-01,  3.5923e-01,
        -1.2338e-01,  6.5812e-03,  3.8730e-01,  2.3091e-01,  1.4636e-01,
         4.6668e-01,  2.6290e-01,  1.6372e-01,  6.9316e-02,  1.4958e-01,
        -5.9775e-01,  2.1966e-01, -1.5152e-01, -7.7528e-03,  2.8760e-02,
         8.2232e-01, -2.1874e-01,  4.2410e-02,  8.8731e-01,  9.9789e-02,
        -6.2738e-01, -4.2279e-01,  9.3123e-03, -3.0431e-01, -2.4775e-01,
        -9.0133e-02,  1.4442e+00,  2.2802e-01,  4.4458e-01, -3.1874e-01,
        -6.9717e-01,  6.2764e-01, -3.7871e-02, -6.9396e-02,  1.3187e-01,
        -1.4630e-01, -9.5384e-02, -4.0134e-02, -1.1

In [9]:
from transformers import BertJapaneseTokenizer

tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')


In [10]:
tknz.tokenize("私は犬が好き")

['私', 'は', '犬', 'が', '好き']